In [26]:
!pip install pandas
!pip install numpy
!pip install torchvision
!pip install scikit-learn
!pip install tensorflow

In [34]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [41]:
df_traindata = pd.read_csv("../data/train.csv")

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_traindata['sentiment'].tolist())

tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(df_traindata['text'].tolist())
sequences = tokenizer.texts_to_sequences(df_traindata['text'].tolist())
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

train_texts, validation_texts, train_labels, validation_labels = train_test_split(
    padded_sequences.tolist(), 
    labels.tolist(), 
    test_size=0.2, 
    random_state=42
)
train_texts = np.array(train_texts)
train_labels = np.array(train_labels)
validation_texts = np.array(validation_texts)
validation_labels = np.array(validation_labels)

df_testdata = pd.read_csv("../data/test.csv")
test_texts = df_testdata['text'].tolist()
test_labels = df_testdata['sentiment'].tolist()

In [ ]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=64),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_texts, train_labels, epochs=10, batch_size=32, validation_data=(validation_texts, validation_labels))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

Epoch 1/10
 772/4500 ━━━━━━━━━━━━━━━━━━━━ 1:32 25ms/step - accuracy: 0.5793 - loss: 0.6824